# CoNLL 2003

In [54]:
from collections import Counter
import numpy as np
import pandas as pd
import string
import torch
import pickle
from tqdm.notebook import tqdm

In [2]:
%%time
cn = pd.read_csv('conceptnet_en.csv')[['subject', 'relation', 'object']]

CPU times: user 4.94 s, sys: 522 ms, total: 5.46 s
Wall time: 5.46 s


In [3]:
paths = {'train': 'data/eng.train', 'dev': 'data/eng.testa', 'test': 'data/eng.testb'}

In [4]:
paths = {'train': 'data/eng.train', 'dev': 'data/eng.testa', 'test': 'data/eng.testb'}
data, vocabulary = {}, {}

for split, path in paths.items():
    print(split)
    data[split] = []
    vocabulary[split] = set()
    
    with open(path) as file:
        tokens, labels = [], []
        
        for line in file:
            if line.startswith('-DOCSTART-'):
                continue
            elif line.strip():
                token, _, _, label = line.split(' ')
                vocabulary[split].add(token)
                tokens.append(token)
                labels.append(label[:-1])
                
            elif tokens and labels:
                data[split].append((tokens, labels))
                tokens, labels = [], []

train
dev
test


In [5]:
all_vocabulary = set(vocabulary['train']).union(set(vocabulary['dev']).union(set(vocabulary['test'])))

In [28]:
print([w for w in all_vocabulary if not w.isalpha()][:100])

['834405', '13.', '1996-12-06', '.8787', 'date:August', '1:55.281', '203', 'razzle-dazzle', '6.0-percent', '6000', 'close-knit', '1:54.984', '7.83', '1,806', '+0.9;+23.6', 'Owen-Jones', '8.3', 're-election', '9,306,000', '3404.71', '480.72', '0:09', 'U.S', '100=1992', '597', '15-6', '30.', '1:51.04', '.444', 'anti-guerrilla', 'asset-St', '1966-76', '77.06', '4:05.00', '9.', '257', '48-43', '45.34', 'oil-based', '27,600', '12-2', '+65-8703086', 'volcano-hit', '1,037', '3:42.852', '38:34.999', '30,300-30,400', '0.65', '.578', '1997-98', 'Above-normal', '13-year', '68.44', '2,389', '3.89', '11.', 'first-inning', '21.25', '36-year-old', 'Mont-sur-Marchienne', '12-match', 'Hindu-Moslem', '222,457', '67.5', '68.80', '28,000', '5,120', '59440', '87.45', '55-11-2324411', 'no-nonsense', 'KwaZulu-Natal', 'well-oiled', '5522', 'PRO-LEAGUE', '42-36', 'higher-than-average', '6mth', '9.77', '31.44', '11-6', 'F.W.', '48.61', '10-0-52-0', '26:28.2', '33-26', '3.92', '1990', 'left-footed', '13.45', '14

In [7]:
print([w for w in entities['ORG'] if not w.isalpha()])

NameError: name 'entities' is not defined

In [13]:
entities

NameError: name 'entities' is not defined

In [14]:
print('Split', '\t', '|N|', '\t', 'Unique')
print('---------------------------')
N = 0
word_freq = {}
for s in data:
    word_freq[s] =  Counter([])
    for d in data[s]: word_freq[s] += Counter(d[0])
    print(s, '\t', str(sum(word_freq[s].values())) + '\t', len(word_freq[s].keys()))
    N += sum(word_freq[s].values())
all_words_freq = word_freq['train'] + word_freq['dev'] + word_freq['test']
print('All', '\t', str(N) + '\t', len(all_vocabulary))

Split 	 |N| 	 Unique
---------------------------
train 	 203621	 23623
dev 	 51362	 9966
test 	 46435	 9488
All 	 301418	 30289


In [15]:
print('Split', '\t', '|V|', '\t', '|v|')
print('---------------------------')
for s in vocabulary:
    print(s, '\t', len(vocabulary[s]), '\t', len(set([t.lower() for t in vocabulary[s]])))
print('All', '\t', str(len(all_vocabulary)) + '\t', len(set([t.lower() for t in all_vocabulary])))

Split 	 |V| 	 |v|
---------------------------
train 	 23623 	 21009
dev 	 9966 	 9002
test 	 9488 	 8548
All 	 30289	 26869


In [16]:
print('Number of tokens unique to the devset:',len(set([t for t in vocabulary['dev'] if t not in vocabulary['train']])))
print('Number of tokens unique to the testset:', len(set([t for t in vocabulary['test'] if t not in vocabulary['train']])))

Number of tokens unique to the devset: 3260
Number of tokens unique to the testset: 3693


In [17]:
entities = {'PER': set(), 'ORG': set(), 'LOC': set(), 'MISC': set()}
# entities = set()

for split in data:
    for tokens, labels in data[split]:
        for token, label in zip(tokens, labels):
            if label != 'O':
                entities[label.split('-')[-1]].add(token)

In [18]:
print('Split', '\t', '|E|', '\t', '|e|')
print('---------------------------')
for l in entities:
    print(l, '\t', len(entities[l]), '\t', len(set([e.lower() for e in entities[l]])))

Split 	 |E| 	 |e|
---------------------------
PER 	 5555 	 5404
ORG 	 3235 	 3005
LOC 	 1868 	 1503
MISC 	 1343 	 1211


In [19]:
all_entities_lower = set()
for l in entities:
    for e in entities[l]:
        all_entities_lower.add(e.lower())

In [20]:
len(all_entities_lower)

10108

In [21]:
all_cn_nodes = set(cn.subject.unique())
len(all_cn_nodes)

1165189

In [22]:
entity_nodes_in_cn = [e for e in all_entities_lower if e in all_cn_nodes]
len(entity_nodes_in_cn)

5558

In [23]:
train_words_in_cn = [e for e in [w.lower() for w in vocabulary['train']] if e in all_cn_nodes]
len(train_words_in_cn)

15747

In [24]:
outside_nodes = [e for e in all_entities_lower if e not in all_cn_nodes]

In [29]:
outside_nodes[:20]

['clignet',
 'keun',
 'nabil',
 'bagwell',
 'stieglmair',
 'caltex',
 'quequen',
 'roest',
 'tramacchi',
 'gulyayeva',
 'kareda',
 'bergamo-based',
 'batigol',
 'eliud',
 'jose-maria',
 'frontieres',
 'baeron',
 'hasina',
 'zdf',
 'danciulescu']

In [34]:
cn_isa = cn[(cn.subject.isin(final_vocab)) & (cn.relation == 'isa') & (cn.object.isin(target_classes))].reset_index()
cn_isa.head()

NameError: name 'final_vocab' is not defined

In [31]:
train_word_with_isa = cn_isa[cn_isa.subject.isin(train_words_in_cn)].subject.unique()
len(train_word_with_isa)

NameError: name 'cn_isa' is not defined

In [32]:
freq = Counter(cn_isa[cn_isa.subject.isin(train_words_in_cn)].object.values)
pd.Series([v for w, v in freq.most_common(100)], index=[w for w, v in freq.most_common(100)]).plot(figsize=(12, 5))

NameError: name 'cn_isa' is not defined

In [ ]:
print([w for w, v in freq.most_common(100)])

In [ ]:
for f in [1, 2, 3]:
    rare_words = [w for w in all_words_freq if all_words_freq[w] <= f]
    print('freq <=', f, ':', len(rare_words))

In [30]:
entity_nodes_with_isa = cn_isa[cn_isa.subject.isin(entity_nodes_in_cn)].subject.unique()
len(entity_nodes_with_isa)

NameError: name 'cn_isa' is not defined

In [ ]:
all_words_freq.most_common(10)23623 	 21009

# Building the Dataset

In [35]:
window_size = 3

In [36]:
nonalpha = [w.lower() for w in vocabulary['train'] if len(w) == 1 and not w.isalpha()]
final_vocab = sorted(set([w.lower() for w in vocabulary['train'] if w not in nonalpha]))
word_to_idx = {w:i for i, w in enumerate(final_vocab + ['[UNK]'])}

In [37]:
word_to_idx['[UNK]']

20978

In [38]:
target_classes = sorted( ['person', 'city', 'country', 'book', 'given_name', 'company', 'musical_artist', 
                          'state', 'film', 'human', 'person_with_nationality', 'album', 'software', 
                          'administrative_region', 'tangible_thing', 'band', 'activity',  'people', 
                          'capital', 'organization', 'weapon', 'magazine', 'river', 'part', 'device', 'structure', 
                          'place', 'town', 'event', 'artifact', 'african', 'area', 'disease', 'soccer_player',
                          'station', 'ethnic_group', 'agent_non_geographical', 'region', 'location', 'asian'] )
class_to_idx = {c:i for i, c in enumerate(target_classes)}
print('Number of properties:', len(target_classes))

Number of properties: 40


In [40]:
cn_isa = cn[(cn.subject.isin(final_vocab)) & (cn.relation == 'isa') & (cn.object.isin(target_classes))].reset_index()
cn_isa.head()

,index,subject,relation,object
0,1087076,11,isa,film
1,1087113,15,isa,band
2,1087128,16,isa,weapon
3,1087155,1984,isa,film
4,1087156,1984,isa,album


In [67]:
! ls

20M_classifier_all_available_features.ipynb   hparams.yaml
conceptnet_en.csv			      lightning_logs
data					      onehot_pytorch_lightning.ipynb
edge_list_generation.ipynb		      prepare_dataset.ipynb
graph_embeddings_generation.ipynb	      README.md
GraphNER_binary_representation_pytorch.ipynb  runs


In [70]:
# cn_isa.to_csv('data/conceptnet_isa.csv')
cn_isa.head(20)

,index,subject,relation,object
0,1087076,11,isa,film
1,1087113,15,isa,band
2,1087128,16,isa,weapon
3,1087155,1984,isa,film
4,1087156,1984,isa,album
5,1087187,1st,isa,album
6,1087216,201,isa,device
7,1087252,25,isa,weapon
8,1087257,26,isa,weapon
9,1087262,28,isa,weapon


In [41]:
! mkdir data/ws3_40r
! mkdir data/ws3_40r/train
! mkdir data/ws3_40r/dev
! mkdir data/ws3_40r/test

! ls data

mkdir: impossible de créer le répertoire « data/ws3_40r »: Le fichier existe
mkdir: impossible de créer le répertoire « data/ws3_40r/train »: Le fichier existe
mkdir: impossible de créer le répertoire « data/ws3_40r/dev »: Le fichier existe
mkdir: impossible de créer le répertoire « data/ws3_40r/test »: Le fichier existe
eng.testa	   eng.testb	      eng.train		 ws3_40r
eng.testa.openNLP  eng.testb.openNLP  eng.train.openNLP


In [42]:
def encode_token(word_idx, context, 
                 df_props=cn_isa, 
                 word_to_idx=word_to_idx, 
                 class_to_idx=class_to_idx, 
                 window_size=window_size):
    
    voc_size = len(word_to_idx)
    word = context[word_idx]
    
    n_extra = 4 #sparse_data['train']
    is_notincn = 1 if word.lower() in all_cn_nodes else 0 
    is_allcaps = 1 if word == word.upper() else 0
    is_initials = word.count('.') > 0 and (word.count('.') + 1) == len(word.split('.')) # C.J or C.J.
    is_capitalized = 1 if word[0] == word[0].upper() else 0

    vec_size = voc_size * 3 + len(class_to_idx) + n_extra
    
    node  = [word_to_idx[word.lower() if word.lower() in word_to_idx else '[UNK]']]
    left  = [voc_size * 1 + (word_to_idx[w.lower()] if w.lower() in word_to_idx else word_to_idx['[UNK]']) 
             for w in context[max(0, word_idx-window_size):word_idx]]
    right = [voc_size * 2  + (word_to_idx[w.lower()] if w.lower() in word_to_idx else word_to_idx['[UNK]']) 
             for w in context[word_idx+1:min(len(context)-1, word_idx+window_size+1)]]
    props = [voc_size * 3 + class_to_idx[c] for c in df_props[df_props.subject == word.lower()].object.values]
    extra = list(range(vec_size - n_extra, vec_size))
    
    i = torch.LongTensor([node + left + right + props + extra])
    v = torch.FloatTensor([1] * len(node + left + right + props) + [is_notincn, is_allcaps, is_initials, is_capitalized])
    vec = torch.sparse.FloatTensor(i, v, torch.Size([vec_size]))
    
    return vec

In [65]:
len(class_to_idx)

40

In [43]:
voc_size = len(word_to_idx)
voc_size, voc_size * 2, voc_size * 3

(20979, 41958, 62937)

In [44]:
encode_token(2, ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']).to_dense().shape

torch.Size([62981])

In [59]:
sparse_data = {}
for split in data:
    print('\n\nGenerating data for', split.upper())
    sparse_data[split] = []
    for tokens, labels in tqdm(data[split], desc=split):
        for i, token in enumerate(tokens):
            if token in nonalpha: 
                continue
            
            vec = encode_token(i, tokens, window_size=2)
            idx = vec._indices()[0].numpy()
            sample = (vec, labels[i].split('-')[-1])
            # print(' '.join(final_vocab[ix] for ix in idx if ix < voc_size), sample)
            sparse_data[split].append(sample)



Generating data for TRAIN





Generating data for DEV





Generating data for TEST


In [60]:
sparse_data['train'][2]

(tensor(indices=tensor([[ 9846, 29725, 37330, 47920, 61284, 62957, 62964, 62965,
                         62977, 62978, 62979, 62980]]),
        values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.]),
        size=(62981,), nnz=12, layout=torch.sparse_coo),
 'MISC')

In [61]:
! ls data

conll2003_sparse_dev.pickle    eng.testa	  eng.testb.openNLP  ws2_40r
conll2003_sparse_test.pickle   eng.testa.openNLP  eng.train	     ws3_40r
conll2003_sparse_train.pickle  eng.testb	  eng.train.openNLP


In [62]:
pickle.dump(sparse_data['train'], open('data/ws2_40r/conll2003_sparse_train.pickle', 'wb'))
pickle.dump(sparse_data['dev'], open('data/ws2_40r/conll2003_sparse_dev.pickle', 'wb'))
pickle.dump(sparse_data['test'], open('data/ws2_40r/conll2003_sparse_test.pickle', 'wb'))

In [71]:
pickle.dump(final_vocab + ['[UNK]'], open('data/ws2_40r/vocab_list.pickle', 'wb'))